# Json Encoding and Decoding
- **Encoding:** Converts python native data(dict) type into json.
- **Decoding:** Converts json data type into python native data type(dict).

In [1]:
class Person:
    
    def __init__(self, fname, lname, age):
        if age <=18:
            raise ValueError("Age must be greater than or equal 18.")
        self.fname = fname
        self.lname = lname
        self.age = age
        
    def to_dict(self):
        return {
            "fname": self.fname,
            "lname": self.lname,
            "age": self.age
        }
    
    def __repr__(self):
        return f"{self.fname} {self.lname} (age={self.age})"

class Item:
    
    def __init__(self, item_id, item_name, unit_price):
        if unit_price <=0:
            raise ValueError("Unit price must be greater than zero.")
        
        self.item_id = item_id
        self.item_name = item_name
        self.unit_price = unit_price
    
    def to_dict(self):
        return {
            "item_id": self.item_id,
            "item_name": self.item_name,
            "unit_price": self.unit_price
        }
        
    def __repr__(self):
        return f'{self.item_name}(qty={self.qty}, unit_price={self.unit_price})'

    
class Order:
    
    def __init__(self, order_id, person):
        if not isinstance(person, Person):
            raise TypeError("Unsupported person assignment.")
        self.order_items = []
        self.order_id = order_id
        self.person=person
        
    def add_item(self, item, qty):
        if qty <=0:
            raise valueError("Quantity must greater or equal to 1.")
        if isinstance(item, Item):
            self.order_items.append((item, qty))
        else:
            raise TypeError("Unsupported Item addition")
    
    def order_item_to_dict(self):
        ord_items = []
        for item in self.order_items:
            ord_items.append(dict(**item[0].to_dict(), qty=item[1]))
        return ord_items

In [2]:
ramesh = Person("Ramesh", "Shrestha", 20)

ramesh_order = Order(2599, ramesh)

item1 = Item(101, "Item1", 2000)
item2 = Item(102, "Item2", 5000)

ramesh_order.add_item(item1, 2)
ramesh_order.add_item(item2, 1)

# OrderEncoder
- Order object to dictionary

In [3]:
import json

class OrderEncoder(json.JSONEncoder):
    
    def default(self, obj):
        if not isinstance(obj, Order):
            raise TypeError("Incompatible Order Type")

        order_data = {
            'id': obj.order_id,
            'person': obj.person.to_dict(),
            'order_items': obj.order_item_to_dict(),
            'type': obj.__class__.__name__
        }
        
        return order_data

In [4]:
orderJSON = OrderEncoder().encode(ramesh_order) # json.dumps(ramesh_order, cls=OrderEncoder)
orderJSON

'{"id": 2599, "person": {"fname": "Ramesh", "lname": "Shrestha", "age": 20}, "order_items": [{"item_id": 101, "item_name": "Item1", "unit_price": 2000, "qty": 2}, {"item_id": 102, "item_name": "Item2", "unit_price": 5000, "qty": 1}], "type": "Order"}'

**Write JSON to file**
- ``json.dump(<dict>, <fp>, cls=<EncoderClass>, indent=<int>)``

In [5]:
with open('order.json', 'w') as file:
    json.dump(ramesh_order, file, cls=OrderEncoder, indent=4)

# Order Decoder
- Converts Order json data to Order Object.

In [6]:
def json_to_order_obj(obj):
    """:
       @params: 
           type<obj> : dict
    """
    if obj['type'] != "Order":
        raise TypeError("Invalid object. Please esure the json type corresponds to Order object.")
    
    
    person = Person(obj['person']['fname'], obj['person']['lname'], obj['person']['age'])
    order_id = obj['id']
    order = Order(order_id, person)
    
    order_item = []
    
    for item in obj['order_items']:
        item_obj = Item(item['item_id'], item['item_name'], item['unit_price'])
        qty = item['qty']
        order.add_item(item_obj, qty)
    
    
    return order

In [12]:
with open('order.json') as fp:
    order = json_to_order_obj(json.load(fp))